In [15]:
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import get_scorer
from sklearn.feature_selection import SelectKBest, f_classif

import category_encoders as ce

import warnings



In [27]:
df, target = pd.read_csv('titanic.csv'), 'Survived'
# df, target = pd.read_csv('EmployeeAttrition.csv'), 'Attrition'
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
X = df.drop(target, axis=1)
y = df[target]

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

y = preprocessing.LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_features = len(numeric_features) + len(categorical_features)

In [34]:
encoder_list = [
    ce.backward_difference.BackwardDifferenceEncoder, 
    ce.basen.BaseNEncoder,
    ce.binary.BinaryEncoder,
    ce.cat_boost.CatBoostEncoder,
    ce.hashing.HashingEncoder,
    ce.helmert.HelmertEncoder,
    ce.james_stein.JamesSteinEncoder,
    ce.one_hot.OneHotEncoder,
    ce.leave_one_out.LeaveOneOutEncoder,
    ce.m_estimate.MEstimateEncoder,
    ce.ordinal.OrdinalEncoder,
    ce.sum_coding.SumEncoder,
    ce.target_encoder.TargetEncoder,
    ce.woe.WOEEncoder
]

for encoder in encoder_list:
    
    numeric_transformer = Pipeline(steps=[
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('cat_encoder', encoder())
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    pipe = Pipeline(
        steps=[
            ('preprocessor', preprocessor),
            ('feature_selection', SelectKBest(f_classif, k=int(0.9*num_features))),
            ('classifier', RandomForestClassifier(n_estimators=250, n_jobs=-1))
        ]
    )
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        model = pipe.fit(X_train, y_train)

        print(f"{encoder.__name__:>30}, {get_scorer('roc_auc')(model, X_test, y_test)}")

     BackwardDifferenceEncoder, 0.8510070580134275
                  BaseNEncoder, 0.8255867332300453
                 BinaryEncoder, 0.8264187754633614
               CatBoostEncoder, 0.8409651689906468
                HashingEncoder, 0.7212371607276067
                HelmertEncoder, 0.8286853732713605
             JamesSteinEncoder, 0.7884030527342629
                 OneHotEncoder, 0.8331324955528778
            LeaveOneOutEncoder, 0.8383829689562174
              MEstimateEncoder, 0.7887473460721868
                OrdinalEncoder, 0.8610489470362082
                    SumEncoder, 0.833362024444827
                 TargetEncoder, 0.86796350490618
                    WOEEncoder, 0.8703161760486602
